In [1]:
# Import PyTorch
import torch
from torch import nn
import torch.nn.functional as F

# Import torchvision
import torchvision
from torchvision import transforms
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torchmetrics

#Import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# write device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = "cpu"
print('Using {} device'.format(device))

In [ ]:
from PIL import Image


def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return Image.fromarray(tensor)

In [ ]:
### Get a dataset fashion MNIST

# Setup Training data

train_data = datasets.FashionMNIST(
    root = 'tf-knugs/datasets',
    train = True,
    download = True,
    transform = transforms.ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'tf-knugs/datasets',
    train = False,
    download = True,
    transform = transforms.ToTensor()
)



In [ ]:
# Setup data loaders
BATCH_SIZE = 32

train_dataloader = DataLoader(
    dataset= train_data,
    batch_size = BATCH_SIZE,
    shuffle=True,
)

test_dataloader = DataLoader(
    dataset= test_data,
    batch_size= BATCH_SIZE,
    shuffle= True
)

In [ ]:
print(f"DataLoaders:{train_dataloader, test_dataloader} ")
print(f"Length of train_dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"Length of test_dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}...")

In [ ]:
print(len(train_data))
print(len(train_dataloader)*BATCH_SIZE)

In [ ]:
#Check out whats inside the training data loader

train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

In [ ]:
train_data.classes

In [ ]:
# show sample
# torch.manual_seed(42)
class_names = train_data.classes
random_idx = torch.randint(0, len(train_features_batch), size=[1]).item()
img, label = train_features_batch[random_idx], train_labels_batch[random_idx]
plt.imshow(img.squeeze(), cmap="gray")
plt.title(class_names[label])
plt.axis(False)


In [ ]:
from torch import nn
class FashionMNISTModelV0(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(), # neural networks like their inputs in vector form
            nn.Linear(in_features=input_shape, out_features=hidden_units), # in_features = number of features in a data sample (784 pixels)
            nn.Linear(in_features=hidden_units, out_features=output_shape)
        )

    def forward(self, x):
        return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)

# Need to setup model with input parameters
model_0 = FashionMNISTModelV0(input_shape=784, # one for every pixel (28x28)
    hidden_units=10, # how many units in the hiden layer
    output_shape=len(class_names) # one for every class
)
model_0.to(device) # keep model on CPU to begin with

In [ ]:
# Import accuracy metric
from helper_functions import accuracy_fn # Note: could also use torchmetrics.Accuracy(task = 'multiclass', num_classes=len(class_names)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.1)

### Creating a function to time our expiriment

Machine learning is very expirimental.

Two main things we want ot keep track of is:
1. Model's performance (loss, acc)etc
2. Models spped or how fast it runs

In [ ]:
from timeit import default_timer as timer
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format).
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [ ]:

def print_shape_device(*args):
    """
    This function prints the shape, data type and device of the given tensors or numpy arrays.

    Parameters:
    *args (torch tensor or numpy array): Variable length argument list of tensors or arrays.

    Returns:
    None
    """
    for arg in args:
        if torch.is_tensor(arg):
            print(f"Shape: {arg.shape} | Device: {arg.device}| Type : {arg.dtype} | torch tensor")
        elif isinstance(arg, np.ndarray):
            print(f"Shape: {arg.shape} | Device: CPU | Type : {arg.dtype} | numpy array")
        else:
            print("Input type not supported. Please provide a torch tensor or a numpy array.")

### Creating a training loop and trainin a model pn batches of data

1. Loop throung epoch.
2. Loop through training batches, perform training steps , calculate the train loss *per batch*.
3. Loop through testing batches, perform tetsing steps, claculate the test loos *per batch*.
4. Print out whats happening.
5. Time is all (for analytics and fun).

In [ ]:
def train_step(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn : torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_fn,
    device:torch.device = device
):
    """
    Trains your model : use within a training loop

    Args:
        model (torch.nn.Module): _description_
        data_loader (torch.utils.data.DataLoader): _description_
        loss_fn (torch.nn.Module): _description_
        optimizer (torch.optim.Optimizer): _description_
        accuracy_fn (_type_): _description_
        device (torch.device, optional): _description_. Defaults to device.
    """

    ### Training
    train_loss, train_acc = 0, 0

    #Put model in Training mode
    model.train()

    # Add a loop to loop through training batches
    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model_0(X)

        # 2. Calculate loss (per batch)
        loss = loss_fn(y_pred, y)
        train_loss += loss # accumulatively add up the loss per epoch
        train_acc += accuracy_fn(y_true= y,
                                 y_pred= y_pred.argmax(dim=1))

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Print out how many samples have been seen
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(data_loader.dataset)} samples")

    # Divide total train loss and accuracy by length of train dataloader (average loss and accuracy per batch per epoch)
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train Loss : {train_loss: .5f} | Train acc : {train_acc: .2f}\n")


In [ ]:
def test_step(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn : torch.nn.Module,
    accuracy_fn,
    device:torch.device = device
):
    """
    Tests the model on test data set

    Args:
        model (torch.nn.Module): _description_
        data_loader (torch.utils.data.DataLoader): _description_
        loss_fn (torch.nn.Module): _description_
        accuracy_fn (_type_): _description_
        device (torch.device, optional): _description_. Defaults to device.
    """
    ### Testing
    # Setup variables for accumulatively adding up loss and accuracy
    test_loss, test_acc = 0, 0

    #Put the model in eval mode
    model.eval()

    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss (accumatively)
            test_loss += loss_fn(test_pred, y) # accumulatively add up the loss per epoch

            # 3. Calculate accuracy (preds need to be same as y_true)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

        # Calculations on test metrics need to happen inside torch.inference_mode()
        # Divide total test loss by length of test dataloader (per batch)
        test_loss /= len(data_loader)

        # Divide total accuracy by length of test dataloader (per batch)
        test_acc /= len(data_loader)

    ## Print out what's happening
    print(f"\nTest loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")

In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm
from helper_functions import accuracy_fn

# Set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_gpu = timer()


# Set the number of epochs (we'll keep this small for faster training times)
epochs = 3

# Create training and testing loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")

    #Train
    train_step(
        model= model_0,
        data_loader= train_dataloader,
        loss_fn= loss_fn,
        optimizer= optimizer,
        accuracy_fn= accuracy_fn,
        device= device
    )

    #test
    test_step(
        model= model_0,
        data_loader= test_dataloader,
        loss_fn= loss_fn,
        accuracy_fn= accuracy_fn,
        device = device
    )


# Calculate training time
train_time_end_on_gpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_gpu,
                                           end=train_time_end_on_gpu,
                                           device=str(next(model_0.parameters()).device))

In [ ]:
# make predicitions and get model results

def eval_model(model: torch.nn.Module,
               dataloader: DataLoader,
               loss_fn: torch.nn.Module,
               accuracy_fn
):
    """Returns a dictonary condataining prediciton parameter of a module

    Args:
        model (torch.nn.Module): model to evaluate
        dataloader (DataLoader): a dataloader that loads data by batches
        loss_fn (torch.nn.Module): function that calculates loss
        accuracy_fn (_type_): function that claculates accuracy
    """
    loss, acc = 0,0
    model.eval()
    with torch.inference_mode():
        for X,y in tqdm(dataloader):
            X, y = X.to(device), y.to(device)
            # Make predicitons
            y_pred = model(X)

            # Accumulate the loss and accuracy values
            acc += accuracy_fn(y_true= y, y_pred= y_pred.argmax(dim=1))
            loss = loss_fn(y_pred, y)

        #Scale loss and acc to find the average loss/acc per batch
        loss /= len(dataloader)
        acc /= len(dataloader)

    return {
        "model_name" : model.__class__.__name__,
        "model_loss": loss.item(),
        "model_acc": acc
    }

In [ ]:
# Use the function to calculate
model0_results = eval_model(
    model=model_0,
    dataloader= test_dataloader,
    accuracy_fn= accuracy_fn,
    loss_fn= loss_fn
)
print(model0_results)

### Creatiung a convolutional neural network

In [ ]:
class FashionMNISTModelv2(nn.Module):
    """
    Model architechture that replicates the TinyVGG
    """
    def __init__(self,
                 input_shape: int,
                 hidden_units: int,
                 output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(
                in_channels= input_shape,
                out_channels=hidden_units,
                kernel_size= 3,
                stride = 1,
                padding=1), #Setting hyperparameters for 2d data
            nn.ReLU(),
            nn.Conv2d(
                in_channels= hidden_units,
                out_channels= hidden_units,
                kernel_size=3,
                padding=1,
                stride=1,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                padding=1,
                stride=1
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=hidden_units,
                out_channels=hidden_units,
                kernel_size=3,
                padding=1,
                stride=1
            ),
            nn.ReLU(),
            nn.MaxPool2d( kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(
                in_features= hidden_units*7*7,
                out_features= output_shape
            )
        )

    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

In [ ]:
torch.manual_seed(42)
model_2 = FashionMNISTModelv2(
    input_shape= 1,
    hidden_units= 10,
    output_shape= len(class_names)
).to(device)

### Train the CNN model

#### Setup a loss and optimizer funciton


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    params=model_2.parameters(),
    lr=0.1
)

In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm
from helper_functions import accuracy_fn

# Set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_gpu = timer()


# Set the number of epochs (we'll keep this small for faster training times)
epochs = 3

# Create training and testing loop
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n-------")

    #Train
    train_step(
        model= model_2,
        data_loader= train_dataloader,
        loss_fn= loss_fn,
        optimizer= optimizer,
        accuracy_fn= accuracy_fn,
        device= device
    )

    #test
    test_step(
        model= model_2,
        data_loader= test_dataloader,
        loss_fn= loss_fn,
        accuracy_fn= accuracy_fn,
        device = device
    )


# Calculate training time
train_time_end_on_gpu = timer()
total_train_time_model_0 = print_train_time(start=train_time_start_on_gpu,
                                           end=train_time_end_on_gpu,
                                           device=str(next(model_2.parameters()).device))